In [1]:
import os
from PIL import Image
import pytesseract
import re
from pdf2image import convert_from_path

import logging
import sys

from os import listdir
from os.path import isfile, join

import pandas as pd
import numpy as np

In [2]:
mypath = '../../data/Logs2019/'
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [15]:
year_str = '19-'
current_date = '01/01/2019'

parsed_addresses = pd.DataFrame(columns = ["call_number","street_number","street_name"])

i_entry = 0
parsed_pages = []
last_call = 0
last_incident = []

for i in range(len(files)):
    with open('../../data/Logs2019/page_{}.txt'.format(i+1),'r') as infile:
        page_text = infile.read()
    
    page_text = page_text.replace("~","-")
    page_string_list = last_incident + page_text.replace(" ","_").split("__")
    calls_idx = [i for i in range(len(page_string_list)) if re.search('19-(?=[0-9])',page_string_list[i]) != None] + [-1]

    for i in range(len(calls_idx)-2):

        incident = page_string_list[calls_idx[i]:calls_idx[i+1]]
        
        if "(WIA" in incident:
            incident.remove("(WIA")
            
        call = page_string_list[calls_idx[i]]
        
        # Check that call has the expected format.
        if re.search(r'^19-?[0-9]+\Z',call):
            
            # Will pass if call is not sequential. 
            if np.abs(int(call.split("-")[1]) - last_call == 0) > 10:
                pass
            else:
                
                # Get standalone house numbers from list of strings.
                street_num = None
                num_list = [re.search(r'[\[,\(,\{,E,I,i]?[W,H,M,N]?[E,I,i,T]A_?[0-9,A-Z]+[\],\),\}]',i
                            )[0] for i in incident if re.search(
                                      r'[\[,\(,\{,E,I,i]?[W,H,M,N]?[E,I,i,T]A_?[0-9,A-Z]+[\],\),\}]',i)]
                if len(num_list) == 1:
                    street_num = num_list[0].strip("([{}])").split("_")[-1]
                
                # If house number is unattached to street, remove it from the list.
                for n in num_list:
                    if n in incident:
                        incident.remove(n)
                
                # Get street name.
                street_name = None
                loc_series = pd.Series(incident)[pd.Series(incident).str.match(r'Locat*')]
                if len(loc_series) > 0:
                    loc_idx = loc_series.index[0]
                    if loc_idx != len(incident) - 1:
                        
                        # Set street to be the entry directly after Locat*
                        street = incident[loc_idx + 1].upper().replace("8","S").strip("_").replace("&","S")
                        street_name = street
                        
                        # Check whether street name is preceeded by house number.
                        num_str = re.search(r'[\],\},\)]_',street)
                        
                        # Split house number from street name.
                        if num_str:
                            if len(street.split(num_str[0])) > 1:
                                street_name = street.split(num_str[0])[-1]
                                
                            street_num = street.split(num_str[0])[0].split("_")[-1]
                        
                        # Preliminary string cleaning.
                        if street_name:
                            street_name = street_name.upper().strip("_").strip(":").replace(
                                "8","S").replace("3","S").replace("5","S").replace(
                                "&","S").replace("SST","ST").replace("S'T","ST").replace(
                                "RDB","RD").replace("RP","RD").replace("FP","RD").replace(
                                "$","S").replace("RDO","RD").replace("_RB","_RD").replace(
                                "_FD","_RD").replace(".","").replace(";","")
                            
                        if street_num:
                            street_num = street_num.replace(",","").replace("S","5").replace("$","5")
                            numbers = sum(c.isdigit() for c in street_num)
                            letters = sum(c.isalpha() for c in street_num)
                            if numbers < letters:
                                street_num = None

                
                parsed_addresses.loc[len(parsed_addresses)] = [call, street_num, street_name]

                # Catch page breaks.
                last_call = int(call.split("-")[1])
    last_incident = page_string_list[calls_idx[-2]:calls_idx[-1]]



In [16]:
# Fix annoying leftover typos.

parsed_addresses["street_name"].replace({

    'BLATR_RD':'BLAIR_RD',
    
    'BULELEY_ST':'BULKLEY_ST',
    'BOLKLEY_ST':'BULKLEY_ST',
    'BULKELEY_ST':'BULKLEY_ST',
    
    '(WIA_SSSS_CHURCH_ST':"CHURCH_ST",
    
                'COLB_SPRING_RD':'COLD_SPRING_RD',
                'COLD_SPRING':'COLD_SPRING_RD',
                'COLD_SFRING_RD':"COLD_SPRING_RD",
                'CGLD_SPRING_RD':'COLD_SPRING_RD',
                'COLD_SPRING_RED':'COLD_SPRING_RD',
                'COLG_SPRING_RD':'COLD_SPRING_RD',
                'COLD_SPRING_ED':'COLD_SPRING_RD',


                'PRENIER_AVE':'FRENIER_AVE',
                'FRENTER_AVE':'FRENIER_AVE',
    'FRENITER_AVE':'FRENIER_AVE',


                'HARWOOB_ST':'HARWOOD_ST',
                'HARHOOD_ST':'HARWOOD_ST',
                'HARWOCD_ST':'HARWOOD_ST',
                'HARWCCD_ST':'HARWOOD_ST',
                'HARHWOOD_ST':'HARWOOD_ST',
                'RARWOOD_ST':'HARWOOD_ST',
                'BARWOOD_ST':'HARWOOD_ST',
    
    "BRAWTHORNE_RD":"HAWTHORNE_RD",

                'ROPPER_RD':'HOPPER_RD',
                'HOPFER_RD':'HOPPER_RD',

                'MAZN_ST':'MAIN_ST',
                "MAIN":"MAIN_ST",
                'MATIN_ST':'MAIN_ST',
                'MAIN_AT':'MAIN_ST',
                'BAIN_ST':'MAIN_ST',
                'MATN_ST':'MAIN_ST',

                'NORTH_HOOSAT_RD':'NORTH_HOOSAC_RD',
                'MORTH_HOOSAC_RD':'NORTH_HOOSAC_RD',
                'NORTH_HOCOSAC_RD':'NORTH_HOOSAC_RD',
                'NORTH_HCOSAC_RD':'NORTH_HOOSAC_RD',
                'NORTH_HOOSAC_ED':'NORTH_HOOSAC_RD',
                'NORTH_HOGSAC_RD':'NORTH_HOOSAC_RD',
                'NORTH_HOOSAC_RED':'NORTH_HOOSAC_RD',
                'NORTH_HOCSAC_RD':'NORTH_HOOSAC_RD',
                "NORTH_HOOGSAC_RD":"NORTH_HOOSAC_RD",
                "WORTH_HOOSAC_RD":"NORTH_HOOSAC_RD",
                'NORTH_HGOOSAC_RD':'NORTH_HOOSAC_RD',
                'NORTH_HOOSAC':'NORTH_HOOSAC_RD',
                'NORTH_HCOQSAC_RD':'NORTH_HOOSAC_RD',

                'NORTH_S§T':'NORTH_ST',
                'WORTH_ST':'NORTH_ST',
    
    'ORLONG_RD':'OBLONG_RD',

                'SIMMONDS_RD':'SIMONDS_RD',
                'SIMONDS':'SIMONDS RD',
                'STMONDS_RD':'SIMONDS_RD',
                "SIMONDS_RIT":"SIMONDS_RD",
                'SITMONDS_RD':'SIMONDS_RD',
                'SIMOGNDS_RD':'SIMONDS_RD',
                'SIMONDS_ED':'SIMONDS_RD',
                'SIMCNDS_RD':'SIMONDS_RD',
                'STIMONDS_RD':'SIMONDS_RD',
                'SIMOMDS_RD':'SIMONDS_RD',
                'SIMONDS RD':'SIMONDS_RD',
    
    'SPRING_STP':'SPRING_ST',

                'SOQUTHWORTH_ST':'SOUTHWORTH_ST',
                'SOQUTHHORTH_ST':'SOUTHWORTH_ST',
                'SOCUTHWORTH_ST':'SOUTHWORTH_ST',
                'SOUTHWORTE_ST':'SOUTHWORTH_ST',
                'SOUTHHORTH_ST':'SOUTHWORTH_ST',
    'SOUTHWGETH_ST':'SOUTHWORTH_ST',

                "TACONTIC_TRL":"TACONIC_TRL",
                'TACONIC_TRE':'TACONIC_TRL',
                'TACGNIC_TRL':'TACONIC_TRL',
                'TACONIC_TRAIL':'TACONIC_TRL',
                'TACONEC_TRL':'TACONIC_TRL',
                'TACONIC_TRIL':'TACONIC_TRL',
    
    'THISTLE_PATE':'THISTLE_PATH',

                'UMIT':None,
                'UNIT':None,
                'UNIT_':None,
                'UNIT;':None,
                'TNIT':None,
                'UNIT_;':None,
                'GNIT':None,
                'NIT':None,

                'COLD_SPRING_RD_+_TACONTC_TRL':'COLD_SPRING_RD_+_TACONIC_TRL',
                'NORTH_HOOSAC_RD_+_BRIDGES_RE':'NORTH_HOOSAC_RD_+_BRIDGES_RD',
                'TACONTC_TRL_+_COLD_SPRING_RD':'TACONIC_TRL_+_COLD_SPRING_RD',


                'PURDLE_MT_PASS':'PURPLE_MT_PASS',
                'SCHOCL_ST':'SCHOOL_ST',
                'BRIDGES_KD':'BRIDGES_RD',
                'PINE_COBBLE_ED':'PINE_COBBLE_RD',
                'MAIN_§T_+_STETSON_CT':'MAIN_ST_+_STETSON_CT',
                'TOWN_OF_WILLTAMSTOWN':'TOWN_OF_WILLIAMSTOWN',

                'SWEETBROOR_RD':'SWEETBROOK_RD',

                'HOLLY_LW':'HOLLY_LN',
                'HANCOCK_RE':'HANCOCK_RD',
                'NORTH':'NORTH_ST',
                'BEOOK_ED':'BROOK_RD',
                'A_UNKNOWN_*_*_€':None,
                "NARRATIVE":None,
                'NARRATIVE;':None,
                "|":None,
                'VEHICLE':None,
                '!':None,
    '1S-9220':None
,    'IFIA_10171_STRATTON_RD':'STRATTON_RD',

                }, inplace = True)

parsed_addresses

,call_number,street_number,street_name
0,19-1,249,NORTH_ST
1,19-4,None,COLD_SPRING_RD
2,19-5,None,LONGVIEW_TER
3,19-6,None,FRENIER_AVE
4,19-7,None,HARWOOD_ST
...,...,...,...
10389,19-16055,None,HANCOCK_RD
10390,19-16060,None,None
10391,19-16062,None,None
10392,19-16063,None,None


In [17]:
parsed_addresses.to_csv("../../data/addresses.csv")

In [6]:
incident_counts = pd.DataFrame(columns = ["street_name","incident_count"])
for k,v in parsed_addresses["street_name"].value_counts().to_dict().items():
    incident_counts.loc[len(incident_counts)] = [k,v]
    
incident_counts.to_csv("../../data/incident_counts.csv")

In [10]:
numbers = sum(c.isdigit() for c in "EAD5BORO")
letters = sum(c.isalpha() for c in "EAD5BORO")

numbers < letters

True